In [1]:
# Import packages
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestRegressor
from IPython.display import clear_output

# Set working path
path = '/Users/martinbogaert/Desktop/NBA Data Analysis/nba_ai_awards/'

print('MAKE SURE YOU CHANGE THE WEEK VARIABLE')

MAKE SURE YOU CHANGE THE WEEK VARIABLE


In [2]:
week = 11 # CHANGE THIS
date = datetime.now().strftime('%d %B %Y')
time = datetime.now().strftime('%H:%M:%S')
print(date + ' ; ' + time)

02 January 2023 ; 18:23:00


# Features, model

In [3]:
features = ['VORP','WS','OWS','BPM','PER','PTS','FT','FTA','OBPM','FG','USG%','Seed','MP']#,'GS'] # GS !!!
model = RandomForestRegressor(n_estimators = 100, max_features = 'auto', min_samples_leaf = 2)

# Minimum minutes per game
mp = 10
# Propotion of maximum games played
gp = 1/3

In [4]:
# Load training data
train_data = pd.read_csv(path + 'Algorithm/mvp/mvp_data.csv')

# Load to-be-predicted data
data = pd.read_csv(path + f'Algorithm/weekly data/week_{week}.csv') 

# Load weekly info
with open(path + f'Algorithm/weekly data/week_{week}.txt', 'r') as file:
            date1 = file.readline().splitlines()[0][7:]
            date2 = file.readline().splitlines()[0][7:]
            n = int(file.readline()[12:])
            
season_progress = n / 15
data.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A,roy,smoy
0,A.J. Green,SG,23,MIL,14,0,7.9,1.4,2.6,0.514,...,1.75,112.78,110.93,1.85,1.69,112.84,111.02,1.82,1,1
1,Bobby Portis,PF,27,MIL,36,9,26.3,5.9,11.9,0.492,...,1.75,112.78,110.93,1.85,1.69,112.84,111.02,1.82,0,1
2,Brook Lopez,C,34,MIL,35,35,30.5,5.5,10.9,0.501,...,1.75,112.78,110.93,1.85,1.69,112.84,111.02,1.82,0,0
3,George Hill,PG,36,MIL,30,0,20.0,1.7,3.9,0.427,...,1.75,112.78,110.93,1.85,1.69,112.84,111.02,1.82,0,1
4,Giannis Antetokounmpo,PF,28,MIL,30,30,33.7,11.5,21.5,0.533,...,1.75,112.78,110.93,1.85,1.69,112.84,111.02,1.82,0,0


In [5]:
# Set training data
X_train = train_data[features]
y_train = train_data['Share']

# Set minimum requirements (games and minutes played)
data = data[data['MP'] >= mp] # Minimum of minutes per game
data = data[data['G'] >= season_progress * gp] # At least 1/3rd of max games played

pred = []
for step in range(0, 10) :
    print('Model ' + str(step+1) + '/10 ...')
    clear_output(wait = True)
    # Fit Machine Learning model
    model.fit(X_train, y_train)

    # Predict shares of test data
    pred.append(model.predict(data[features]))

Model 10/10 ...


In [6]:
# Assemble results DataFrame
res = data.assign(Share = [np.array(pred)[:,i].mean() for i in range(0, len(data))])
res = res.sort_values('Share', ascending = False) # Sort values by predictions
res['Rank'] = list(range(1, len(res)+1))
res['week'] = len(res) * [week] # Add week

res = res.reset_index(drop = True)[['Player' ,'Tm', 'PER', 'VORP', 'WS', 'PTS', 'Seed', 'Share']]
res.head()

,Player,Tm,PER,VORP,WS,PTS,Seed,Share
0,Nikola Jokić,DEN,32.4,10.187879,18.139394,25.7,1,0.907948
1,Luka Dončić,DAL,32.5,10.611765,17.605882,34.2,4,0.728498
2,Joel Embiid,PHI,30.3,7.592593,14.577778,33.2,5,0.448962
3,Kevin Durant,BRK,26.6,6.560000,13.354286,29.7,2,0.421634
4,Jayson Tatum,BOS,24.9,5.622857,12.182857,30.9,1,0.417728


In [7]:
res.to_csv(path + f'Results/mvp/results_week_{week}.csv', index = None)